
<center>
    <img src='./assets/images/besm.png' width='150px'>
</center>

In [1]:
%%html
<style>
    .jp-MarkdownOutput {
        direction: rtl;
        text-align: right;
        font-family: 'Vazir', 'B Nazanin', 'B Lotus', 'Arial', 'Verdana'
    }
    h4 {
        padding: 0px;
        margin: 0px;
    }
    div, span {
        font-size: 14px;
    }
</style>

<h3><center>تمرین نخست درس «پردازش زبان‌های طبیعی»</center></h3>
<h4><center> جواد راضی (۴۰۱۲۰۴۳۵۴) </center></h4>
<h4><center> ترم ۴۰۱۲ </center></h4>
<br>



<h2>
    متن انتخابی
</h2>
  

<span>
    در این تمرین، کامنت‌های زیر چند شعر مشهور در وب‌سایت گنجور استخراج شده، و بر روی آن‌ها یکسری پردازش اعمال می‌شود. 
</span>

<h2>
    کاوش متن و استخراج کامنت‌های سایت
</h2>
<div>
    <span>
        برای crawling، از ابزار scrapy در این تمرین استفاده شده‌است.
    </span>
    <span>
        scrapy، یک کتاب‌خانه پایتون است که به واسطه آن می‌توان اطلاعات مورد نظر، نظیر متون،‌تصاویر، و محتوای بخش‌های خاصی از سایت را از آن استخراج نمود. 
    </span>
    <br/>
    <span>
        اسکریپت پایتون زیر، یک کامنت‌های مربوط به چند شعر انتخابی از خیام، حافظ، و پروین اعتصامی را استخراج می‌نماید. کامنت‌های کراول شده، ساختاری nested دارند؛ به عبارت دیگر، برای هر شعر، کامنت‌‌ها خود می‌توانند شامل کامنت‌های پاسخ باشند که در قالب یک آبجکت nested ذخیره می‌شود.
    </span>
</div>

In [2]:
try:
    import scrapy 
except: 
    !pip3 install scrapy
try:
    import crochet
except:
    !pip3 install crochet

In [3]:
# scrape webpage
import scrapy
from scrapy.crawler import CrawlerRunner
# text cleaning
import re
# Reactor restart
from crochet import setup, wait_for
setup()


In [4]:
class GanjoorSpider(scrapy.Spider):
    name = "GanjoorSpider"
    start_urls = [
        'https://ganjoor.net/khayyam/robaee/sh41',
    ]
    custom_settings = {
        'ITEM_PIPELINES': {
            # '__main__.ExtractFirstLine': 1
        },
        'FEEDS': {
            'ganjoor.csv': {
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    def parse(self, response):
        """parse data from urls"""
        for comment in response.css('#comments-block > .ganjoor-comment > blockquote > p:text'):
            yield {'comment': comment.extract()}


class ExtractFirstLine(object):
    def process_item(self, item, spider):
        """text processing"""
        lines = dict(item)["comment"].splitlines()
        first_line = self.__remove_html_tags__(lines[0])

        return {'quote': first_line}

    def __remove_html_tags__(self, text):
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)


@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(GanjoorSpider)
    return d


In [5]:
run_spider()

In [6]:
# import scrapy
#
# class GanjoorSpider(scrapy.Spider):
#     name = "ganjoor_comments"
#     start_urls = [
#         "https://ganjoor.net/khayyam/robaee/sh41"
#     ]
#
#     def parse(self, response):
#         for comment in response.css('.comment'):
#             yield {
#                 'author': comment.css('.author::text').get(),
#                 'text': comment.css('.text::text').get(),
#                 'date': comment.css('.date::text').get(),
#             }
#

# Parsi-IO 

### https://github.com/language-ml/parsi.io

تلاش گروهی متن‌باز برای استخراج اطلاعات فارسی

In [7]:
from parsi_io.modules.address_extractions import AddressExtraction
extractor = AddressExtraction()
extractor.run('آدرس خیابان شهیدبهشتی می‌باشد و برای اطلاعات بیشتر به page.com مراجعه فرمایید')

ModuleNotFoundError: No module named 'parsi_io'

In [ ]:
from parsi_io.modules.number_extractor import NumberExtractor
extractor = NumberExtractor()
extractor.run(" یک صد و بیست میلیون تومان هزینه داشت")

In [ ]:
from parsi_io.modules.time_extractions import TimeExtraction
extractor = TimeExtraction()
extractor.run("روز  ۷ اسفند کتابخانه پانزده خرداد می آیم")

<h2 style='direction:rtl;'> § نمونه‌ای از متن  </h2> 



In [ ]:
import random 
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm

#tqdm.tqdm_notebook()
    
# Path of poems
poem_path = "../../exploring-datasets/literature/iranian"
HAFEZ_end=8384
SAADI_start=9384
mesra_collection = [x.strip() for x in tqdm.notebook.tqdm(codecs.open(F'{poem_path}/qazals/all_qazals_mesra.txt','rU','utf-8').readlines())]

In [ ]:
mesra_collection[0]

<h2 style='direction:rtl;'> § نرمالایز کردن  </h2> 



# Hazm 

### https://github.com/roshan-research/hazm

In [ ]:
from __future__ import unicode_literals
from hazm import *

hazm_normalizer = Normalizer()

mesra_normalized_hazm = [hazm_normalizer.normalize(x) for x in tqdm.notebook.tqdm(mesra_collection)]

# Dadmatools 

### https://github.com/Dadmatech/DadmaTools
### https://aclanthology.org/2022.naacl-demo.13

In [ ]:
from dadmatools.models.normalizer import Normalizer

dadma_normalizer = Normalizer(
    full_cleaning=True,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with="<NUM>",
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with=None,
    replace_home_number_with=None,
)

mesra_normalized_dadma = [dadma_normalizer.normalize(x) for x in tqdm.notebook.tqdm(mesra_collection)]

In [ ]:
random_mesra = random.sample(range(0,len(mesra_collection)),10)
examples = ["روحُ القدس2 ","می توانم"]



df = pd.DataFrame({'original_text':[mesra_collection[idx] for idx in random_mesra]+examples,
                   'hazm_normalized':[mesra_normalized_hazm[idx] for idx in random_mesra]
                       +[hazm_normalizer.normalize(e) for e in examples],
                   'dadma_normalized':[mesra_normalized_dadma[idx] for idx in random_mesra]
                       +[dadma_normalizer.normalize(e) for e in examples]
                  })
df

<h2 style='direction:rtl;'> § جمله‌بندی  </h2> 



In [ ]:
mesra_sentences = [sent_tokenize(x) for x in tqdm.notebook.tqdm(mesra_normalized_hazm)]

In [ ]:
random_mesra = random.sample(range(0,len(mesra_collection)),10)
examples = ["علی به خانه رفت . آنجا دو روز ماند. ","ای.آی مخفف هوش مصنوعی است"]



df = pd.DataFrame({'original_text':[mesra_collection[idx] for idx in random_mesra]+examples,
                   'hazm_text':['---'.join(mesra_sentences[idx]) for idx in random_mesra]
                       +['---'.join(sent_tokenize(e)) for e in examples],
                  })
df

<h2 style='direction:rtl;'> § توکنایزیشن  </h2> 



In [ ]:
mesra_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.notebook.tqdm(mesra_sentences)]

In [ ]:
random_mesra = random.sample(range(0,len(mesra_collection)),10)
examples = [" او خوش‌برخورد است"]



df = pd.DataFrame({'original_text':[mesra_collection[idx] for idx in random_mesra]+examples,
                   'hazm_text':[mesra_tokens[idx] for idx in random_mesra]
                       +[word_tokenize(hazm_normalizer.normalize(e)) for e in examples],
                  })
df

<h2 style='direction:rtl;'> § تحلیل بسامد  </h2> 

In [ ]:
import nltk
nltk.download()

In [ ]:
from nltk import FreqDist
import itertools
import pandas as pd

all_tokens = list(itertools.chain(*itertools.chain(*mesra_tokens)))
hafez_tokens = list(itertools.chain(*itertools.chain(*mesra_tokens[0:HAFEZ_end])))
saadi_tokens = list(itertools.chain(*itertools.chain(*mesra_tokens[SAADI_start::])))

dataframe = {}

for opt in ['hafez', 'saadi', 'all']:
     dataframe[opt] = FreqDist(eval(F"{opt}_tokens")).most_common(25)

freq_analysis = pd.DataFrame(dataframe)    

In [ ]:
freq_analysis

In [ ]:
print ('%-16s' % 'Number of words', '%-16s' % len(all_tokens))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(all_tokens)))
avg=np.sum([len(word) for word in all_tokens])/len(all_tokens)
print ('%-16s' % 'Average word length', '%-16s' % avg)
print ('%-16s' % 'Longest word', '%-16s' % all_tokens[np.argmax([len(word) for word in all_tokens])])

<h2 style='direction:rtl;'> § حذف stop-word  </h2> 

In [ ]:
# Persian Stopwords
# https://github.com/sobhe/hazm/blob/master/hazm/data/stopwords.dat
stop_path = '../../resources/persian_stopwords.txt'
stopwords = [hazm_normalizer.normalize(x.strip()) for x in codecs.open(stop_path,'r','utf-8').readlines()]

In [ ]:
stop_path_poems = '../../resources/persian_stopwords-poems.txt'
stopwords = stopwords + [hazm_normalizer.normalize(x.strip()) for x in codecs.open(stop_path_poems,'r','utf-8').readlines()]

In [ ]:
all_tokens_nonstop = [t for t in tqdm.tqdm(all_tokens) if t not in stopwords]
hafez_tokens_nonstop = [t for t in tqdm.tqdm(hafez_tokens) if t not in stopwords]
saadi_tokens_nonstop = [t for t in tqdm.tqdm(saadi_tokens) if t not in stopwords]

dataframe_nonstop = {}

for opt in ['hafez', 'saadi', 'all']:
     dataframe_nonstop[opt] = FreqDist(eval(F"{opt}_tokens_nonstop")).most_common(50)

freq_analysis_nonstop = pd.DataFrame(dataframe_nonstop)    

<h4 style='direction:rtl;'> برای داده خود لیست stop-word ها را بهبود بدهید </h4> 

In [ ]:
freq_analysis_nonstop

<h2 style='direction:rtl;'> § استفاده از lemmatization, stemming  </h2> 

In [ ]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_lemma_set(tok, opt=1):
    if opt ==1:
        return stemmer.stem(tok)
    if opt ==2:
        return lemmatizer.lemmatize(tok)
    if opt ==3:
        # write your own
        pass

In [ ]:
#<NUM> <DATE> 

In [ ]:
opt = 2

all_tokens_nonstop_lemstem =   [get_lemma_set(t, opt) for t in tqdm.tqdm(all_tokens_nonstop)]
hafez_tokens_nonstop_lemstem = [get_lemma_set(t, opt) for t in tqdm.tqdm(hafez_tokens_nonstop)]
saadi_tokens_nonstop_lemstem = [get_lemma_set(t, opt) for t in tqdm.tqdm(saadi_tokens_nonstop)]

dataframe_nonstop_lemstem = {}

for opt in ['hafez', 'saadi', 'all']:
     dataframe_nonstop_lemstem[opt] = FreqDist(eval(F"{opt}_tokens_nonstop_lemstem")).most_common(50)

freq_analysis_nonstop_lemstem = pd.DataFrame(dataframe_nonstop_lemstem)   
freq_analysis_nonstop_lemstem

<h2 style='direction:rtl;'> § استفاده از POS-tags  </h2> 

In [ ]:
tagger = POSTagger(model='../../resources/postagger.model')

In [ ]:
import copy
dataframe_nonstop_lemstem_advjj = {}

for opt in tqdm.tqdm(['hafez', 'saadi', 'all']):
    selected = []
    for x,y in FreqDist(eval(F"{opt}_tokens_nonstop_lemstem")).most_common(1000):
        pos = tagger.tag([x])[0][1]
        if pos in ['V']:
            selected.append((x,y)) 
    dataframe_nonstop_lemstem_advjj[opt]= copy.deepcopy(selected[0:40])

dataframe_nonstop_lemstem_advjj = pd.DataFrame(dataframe_nonstop_lemstem_advjj)    

In [ ]:
# change the code to get the POS's in the sentences

In [ ]:
dataframe_nonstop_lemstem_advjj

<h2 style='direction:rtl;'> § دیدن سیاق  </h2> 

In [ ]:
hafez_text = nltk.Text(hafez_tokens_nonstop_lemstem)
hafez_text.concordance('آب')

<h2 style='direction:rtl;'> § چانکینگ - عبارت یابی  </h2> 

In [ ]:
for sent in mesra_tokens[0:10]:
    print(tagger.tag(sent[0]))
    print()

In [ ]:
tagger.tag(hazm_normalizer.normalize("حافظ خلوت نشین").split())

In [ ]:
def vasf_ext(sentence):
    grammar = r"""
      NVASF: {<N|Ne><AJ>}
      GHEIDV: {<ADV><V>}
    """
    cp = nltk.RegexpParser(grammar)
    return (cp.parse(sentence))

In [ ]:
tagger.tag(mesra_tokens[100][0])

In [ ]:
vasf_ext(tagger.tag(mesra_tokens[100][0]))

In [ ]:
mosufs = []
gheids = []
for sentence in tqdm.tqdm(mesra_tokens[0:HAFEZ_end]):
    tree=vasf_ext(tagger.tag(sentence[0]))
    for subtree in tree.subtrees():
        if subtree.label() == 'NVASF':
            mosufs.append(subtree.leaves())
        if subtree.label() == 'GHEIDV':
            gheids.append(subtree.leaves())            

In [ ]:
mosuf_texts = [' '.join([x[0],y[0]]) for x,y in mosufs]

In [ ]:
mosuf_texts[0:10]

In [ ]:
results = {}

results['hafez'] = nltk.FreqDist(mosuf_texts).most_common(50)

In [ ]:
mosufs = []
gheids = []
for sentence in tqdm.tqdm(mesra_tokens[SAADI_start::]):
    tree=vasf_ext(tagger.tag(sentence[0]))
    for subtree in tree.subtrees():
        if subtree.label() == 'NVASF':
            mosufs.append(subtree.leaves())
        if subtree.label() == 'GHEIDV':
            gheids.append(subtree.leaves())            

In [ ]:
mosuf_texts = [' '.join([x[0],y[0]]) for x,y in mosufs]
mosuf_texts[0:10]

In [ ]:
results['saadi'] = nltk.FreqDist(mosuf_texts).most_common(50)

In [ ]:
pd.DataFrame(results)

# Keyword Extraction

In [ ]:
#hafez = ' '.join([' '.join(s[0]) for s in mesra_tokens[0:8384]])
#saadi = ' '.join([' '.join(s[0]) for s in mesra_tokens[9384::]])
hafez= ' '.join(hafez_tokens_nonstop)
saadi= ' '.join(saadi_tokens_nonstop)

In [ ]:
import yake
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(hafez)

for kw in keywords:
	print(kw)

In [ ]:
import yake
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(saadi)

for kw in keywords:
	print(kw)

# Dadma Tools

In [ ]:
import dadmatools.pipeline.language as language

# here lemmatizer and pos tagger will be loaded
# as tokenizer is the default tool, it will be loaded as well even without calling
pips = 'tok,lem,pos,dep,ner,chunk,cons,kasreh' 
nlp = language.Pipeline(pips)

# you can see the pipeline with this code
print(nlp.analyze_pipes(pretty=True))

In [ ]:
doc = nlp('مسابقات کشتی در کشور ایران برگزار خواهد شد')

In [ ]:
doc._.ners

In [ ]:
doc._.constituency 

In [ ]:
from spacy import displacy

displacy.serve(doc, style="dep")

# Weighted Levenshtein Distance

In [ ]:
#!pip install -U strsimpy
import pandas as pd
from strsimpy.weighted_levenshtein import WeightedLevenshtein


def insertion_cost(char):
    return 1.0


def deletion_cost(char):
    return 1.0


def substitution_cost(char_a, char_b):
    return 2.0

weighted_levenshtein = WeightedLevenshtein(
    substitution_cost_fn=substitution_cost,
    insertion_cost_fn=insertion_cost,
    deletion_cost_fn=deletion_cost)

In [ ]:
print(weighted_levenshtein.distance('لانه', 'لثه'))

In [ ]:
print(weighted_levenshtein.distance('لانه', 'لاله'))

In [ ]:
print(weighted_levenshtein.distance('لانه', 'لستم'))

In [ ]:
print(weighted_levenshtein.distance('لثه', 'لستم'))